In [1]:
import findspark
findspark.init()
import pyspark

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from operator import add
import time

In [5]:
# Read file
spark=SparkSession.builder.appName('Assignment_6_and_7').getOrCreate()
data=(spark.read.option('header','true').csv('Dataset.csv', sep="," , inferSchema=True)).persist()

In [6]:
data.printSchema()

root
 |-- age: string (nullable = true)
 |-- workclass: string (nullable = true)
 |-- education: string (nullable = true)
 |-- Education_numOfYears: integer (nullable = true)
 |-- marital_status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- hous_per_week: string (nullable = true)
 |-- native_country: string (nullable = true)
 |-- fifty_bool: string (nullable = true)



In [2]:
# Read file
sc = SparkContext("local" , "Apriori")
file =  sc.textFile("Dataset.csv")

print(file.collect())

14,Married,Technical-Working-Class, Husband, White, Male,Normal-Hours, United-States, >50K.', 'Young-Age,Private,HS-Graduates,10,Never-married,Other-Service, Own-child, White, Female,Relaxed-Time-Hours, United-States, <=50K.', 'Old-Age,Goverment,HS-Graduates,9,Divorced,Exec-managerial, Unmarried, Amer-Indian-Eskimo, Male,Relaxed-Time-Hours, United-States, <=50K.', 'Middle-Age,Private,Associates,11,Divorced,Technical-Working-Class, Unmarried, White, Female,Part-Time-Hours, United-States, <=50K.', 'Young-Age,Private,Bachelors,13,Never-married,Technical-Working-Class, Own-child, White, Male,Part-Time-Hours, United-States, <=50K.', 'Middle-Age,Private,Associates,11,Divorced,Other-Service, Not-in-family, White, Female,Part-Time-Hours, United-States, <=50K.', 'Middle-Age,Private,HS-Graduates,9,Married,Lowwer-Working-Class, Husband, White, Male,Over-Time-Hours, United-States, <=50K.', 'Young-Age,Private,HS-Graduates,9,Married,Lowwer-Working-Class, Husband, White, Male,Over-Time-Hours, United-

In [3]:
## Splited items  
lblitems = file.map(lambda line: line.split(','))

print(lblitems.collect())

ciates', '11', 'Married', 'Technical-Working-Class', ' Husband', ' White', ' Male', 'Part-Time-Hours', ' United-States', ' <=50K.'], ['Old-Age', 'Private', 'Bachelors', '13', 'Divorced', 'Exec-managerial', ' Unmarried', ' White', ' Male', 'Normal-Hours', ' United-States', ' >50K.'], ['Middle-Age', 'Private', 'HS-Graduates', '10', 'Married', 'Technical-Working-Class', ' Wife', ' White', ' Female', 'Part-Time-Hours', ' United-States', ' >50K.'], ['Middle-Age', 'Private', 'HS-Graduates', '9', 'Married', 'Lowwer-Working-Class', ' Husband', ' White', ' Male', 'Normal-Hours', ' United-States', ' >50K.'], ['Middle-Age', 'Private', 'HS-Graduates', '10', 'Married', 'Exec-managerial', ' Husband', ' White', ' Male', 'Normal-Hours', ' United-States', ' >50K.'], ['Middle-Age', 'Self-Employed', 'Bachelors', '13', 'Married', 'Exec-managerial', ' Husband', ' White', ' Male', 'Over-Time-Hours', ' United-States', ' <=50K.'], ['Young-Age', 'Private', 'HS-Graduates', '9', 'Never-married', 'Other-Service',

# Now, you can clearly see that every line in the transaction data-set is separated by a comma. And every array in our RDD represents the item lists those are customers’ transactions. We successfully red the file and converted it to a Spark RDD object by considering each transaction that belongs to customers.

##### Apriori algorithm depends on the frequencies of items. Because of that first, we need to obtain frequencies for every single item. These frequencies will be our first support values in table one (as mentioned in the previous section). In order to that, we need to extract every item in RDDs to whole items of the array. We can do that by using the “flatMap” method.

In [4]:
## Whole lines in single array 
wlitems = file.flatMap(lambda line:line.split(','))

print(wlitems.collect())

hite', ' Male', 'Part-Time-Hours', ' United-States', ' <=50K.', 'Middle-Age', 'Private', 'Bachelors', '13', 'Married', 'Exec-managerial', ' Husband', ' White', ' Male', 'Normal-Hours', ' United-States', ' >50K.', 'Middle-Age', 'Private', 'Associates', '11', 'Married', 'Technical-Working-Class', ' Husband', ' White', ' Male', 'Part-Time-Hours', ' United-States', ' <=50K.', 'Old-Age', 'Private', 'Bachelors', '13', 'Divorced', 'Exec-managerial', ' Unmarried', ' White', ' Male', 'Normal-Hours', ' United-States', ' >50K.', 'Middle-Age', 'Private', 'HS-Graduates', '10', 'Married', 'Technical-Working-Class', ' Wife', ' White', ' Female', 'Part-Time-Hours', ' United-States', ' >50K.', 'Middle-Age', 'Private', 'HS-Graduates', '9', 'Married', 'Lowwer-Working-Class', ' Husband', ' White', ' Male', 'Normal-Hours', ' United-States', ' >50K.', 'Middle-Age', 'Private', 'HS-Graduates', '10', 'Married', 'Exec-managerial', ' Husband', ' White', ' Male', 'Normal-Hours', ' United-States', ' >50K.', 'Middl

In [6]:
#### As you can see from the result; All our transaction items are in a single array. Now we can calculate each unique item’s frequency. Do not forget that these frequencies will be our first supports values. If we would be working on a “NumPy” array, finding frequencies would be easy. But, we work on RDDs and that’s why we need to find a way to obtain frequencies by considering the “MapReduce” approach. Solution; First we can convert every item to a “tuple” object and add “1” as a second item of “tuple”. We can sum these values by using the “reduceByKey” (It is like the groupby method in SQL) method. By summing tuple’s second numbers we can get every unique item’s frequency (how many time occurs on customers’ transactions). We will also need to list unique items in the feature sections. So, we can also obtain unique items by using the “distinct” method.

In [5]:
## Unique frequent items in dataset
uniqueItems = wlitems.distinct()

# Add 1 as Tuple
supportRdd = wlitems.map(lambda item: (item , 1))

# Method for sum in reduceByKey method
def sumOparator(x,y):
    return x+y

# Sum of values by key
supportRdd = supportRdd.reduceByKey(sumOparator)

# print(supportRdd.collect()) # Retruns following array 
# [('Apple', 12), ('Mango', 10), ('Banana', 9), ('Coconut', 3), 
#  ('Strawberry', 4), ('Grapes', 2), ('Lemon', 1), ('Raspberry', 9), ('Rassberry', 1)]


# First support values
supports = supportRdd.map(lambda item: item[1]) # Return only support values

#### If you run “supportRdd.collect()” after the above code snippet, you get the first item -support tuples (assume as Table A). As you can see from support values, “Apple” has more frequency than the others. It means the probability of “Apple” purchases is greater than the others. These support values are gotten by considering each item separately. We also need to consider how they occur together in transactions. Thanks to that we can calculate confidence values over the Apriori algorithm. We will see these steps in the next sessions.

##### Min Support Value In order to decide which item-sets will stay in the support tables, we need to define a min support value. We can choose min support value as the minimum frequency that is in the first support values array (table). If any support values in item set arrays are less than min support value, we should remove that item set from that array. If our data hasn’t got many records, min support might be 1. In such cases, we can define min support as 2 or any value more than 1.

In [8]:
# Define minimum support value 
minSupport = supports.min()

# If mininmum support is 1 then replace it with 2 
minSupport = 2 if minSupport == 1 else minSupport

## Filter first supportRdd with minimum support 
supportRdd = supportRdd.filter(lambda item: item[1] >= minSupport )

## Craete base RDD with will be updated every iteration
baseRdd = supportRdd.map(lambda item: ([item[0]] , item[1])) 
print('1 . Table has crated...') 

supportRdd = supportRdd.map(lambda item: item[0])
supportRddCart = supportRdd

1 . Table has crated...


#### In this example, we have a minimum support value that has been found as 1. To get a more consistent result we can set min support as 2. It means that if any item or item set occurs in a transaction less than 2 times we won’t take it into account. As you can see from line 8 we filtered our first item set table (which is single item support values) according to min support value. We also created a “baseRdd” object. “baseRdd” represents our first support values for every single item. This object will be updated with upcoming support values of combinations. We also need to define “supportRdd” that shows only items without support values. We will use it in the next sections in order to create combinations of items.
Getting into the Apriori Algorithm
We only found each item’s support value (frequencies) until this section. Now, we will create an algorithm that generates combinations of items in a while loop. There will be different support tables that will be created in every loop. This while loop will end when there isn’t any combination whose support value more than min support value. This algorithm will control how many times these combinations of items occur together in transaction data-set in every loop and save it in an RDD.
We already created a “supportRdd” object which only includes the first table’s item set without support values (which only have one item in each row). Now, we will use this RDD in a while loop to combine it with unique items in order to create other support tables. However, this RDD will be updated after every loop. For example;
# Fitst supportRDD
([Apple] , [Mango] , [Banana] , [Grapes])
# After first loop = supportRDD 
([Apple,Mango],[Mango,Banana],[Apple,Banana],[Apple,Grapes] ......)
# After second loop = supportRDD
([Apple,Mango,Grapes] , [Apple,Banana,Grapes] ........ )
The algorithm will filter every combination table according to min support value. When there is no item set while loop will end. Besides, we also need to define a function that can find replications inside the combined item set. As mentioned before; sets of (Apple, Mango) and (Mango, Apple) are the same thing for the Apriori algorithm. Because of that, we need to find such patterns and remove one of them. As you can see from the below code snippet, there is a function called “removeReplica”. This function removes such duplicated items after a combination and returns only one of them.

In [9]:
def removeReplica(record):

    if(isinstance(record[0], tuple)):
        x1 = record[0]
        x2 = record[1]
    else:
        x1 = [record[0]]
        x2 = record[1]

    if(any(x == x2 for x in x1) == False):
        a = list(x1)
        a.append(x2)
        a.sort()
        result = tuple(a)
        return result 
    else:
        return x1

### Yeap! We are ready to create our while loop. First, we need to define a variable that we can control item-set length in each loop. It is represented as “c” in the above code snippet. This “c” variable starts from 2. Why? Remember that we already created our first support table as “supportRdd”. So, in the while loop support table will start from c=2. It means the first support table that is going to be created in while loop will have item-sets which has 2 items. In order to create combinations of items, we can use the “cartesian” function that comes with PySpark (at line 6). It is created we will remove duplicated items (at line 7). We will use the “c” variable in order to filter combined items more than the “c” variable (at line 9). However, we also use the “distinct” method to get a unique item-set just in case.

In [ ]:
c = 2 # Combination length 

while(supportRdd.isEmpty() == False):

    combined = supportRdd.cartesian(uniqueItems)
    combined = combined.map(lambda item: removeReplica(item))
  
    combined = combined.filter(lambda item: len(item) == c)
    combined = combined.distinct()

    
    combined_2 = combined.cartesian(lblitems)
    combined_2 = combined_2.filter(lambda item: all(x in item[1] for x in item[0]))
    
    combined_2 = combined_2.map(lambda item: item[0])
    combined_2 = combined_2.map(lambda item: (item , 1))
    combined_2 = combined_2.reduceByKey(sumOparator)
    combined_2 = combined_2.filter(lambda item: item[1] >= minSupport)

    baseRdd = baseRdd.union(combined_2)
    
    combined_2 = combined_2.map(lambda item: item[0])
    supportRdd = combined_2
    print(c ,'. Table has crated... ')
    c = c+1

##### As you can see there are two objects created in each loop as “combined” and “combined_2”. Combined already explained above. “Combined_2”, on the other hand, is a combination of the “combined” variable and each line of the whole data set (each transaction). As you can see from line 14, there is a filtering process that controls each item-set whether item-sets of “combined” are in the data-set. If there is not any such item-set in the data-set it removes that item-set. After all, we also get frequencies by using the “reduceByKey” method and filter them according to min support value. At line 21, we add the final support table to “baseRDD” which has our all support values with item-sets (in theory it is called table). However, we also need to update the “supportRdd” variable with “combined_2” without support values. This process will go on until there isn’t left any item-set in “supportRdd”.
Finally, we can calculate the confidence values. You can use the below codes for calculating confidence values for each combination of baseRdd (whole item-set combinations). There is a “Filter” class that can filter data according to confidence calculation. It also includes the method as “calculateConfidence” that calculates confidences.


In [ ]:

class Filter():

    def __init__(self):
        
        self.stages = 1


    def filterForConf(self, item , total):
        
        if(len(item[0][0]) > len(item[1][0])  ):
            if(self.checkItemSets(item[0][0] , item[1][0]) == False):
                pass
            else:
                return (item)       
        else:
            pass  
        self.stages = self.stages + 1

    # Check Items sets includes at least one comman item // Example command: # any(l == k for k in z for l in x )
    def checkItemSets(self, item_1 , item_2):

        if(len(item_1) > len(item_2)):
            return all(any(k == l for k in item_1 ) for l in item_2)
        else:
            return all(any(k == l for k in item_2 ) for l in item_1)


    def calculateConfidence(self, item):

        # Parent item list
        parent = set(item[0][0])
        
        # Child item list
        if(isinstance(item[1][0] , str)):
            child  = set([item[1][0]])
        else:
            child  = set(item[1][0])
        # Parent and Child support values
        parentSupport = item[0][1]
        childSupport = item[1][1]
        # Finds the item set confidence is going to be found

        support = (parentSupport / childSupport)*100

        return list([ list(child) ,  list(parent.difference(child)) , support ])

        
# Example ((('x10', 'x3', 'x6', 'x7', 'x9'), 1), (('x10', 'x3', 'x7'), 1))
calcuItems = baseRdd.cartesian(baseRdd)

# Create Filter Object
ff = Filter()

#deneme = calcuItems.map(lambda item: lens(item)) 
total = calcuItems.count()

print('# : Aggregated support values preparing for the confidence calculatations')
baseRddConfidence = calcuItems.filter(lambda item: ff.filterForConf(item , total))
print('# : Aggregated support values are ready !')
baseRddConfidence = baseRddConfidence.map(lambda item: ff.calculateConfidence(item))

  
print(baseRddConfidence.collect())

# If you run baseRddConfidence.collect() you can get all confidence values. You can also filter the result which is greater than certain confidence. A few examples from the result is shown below;
[
[['Apple'], ['Mango'], 58.333333333333336], 
 [['Mango'], ['Apple'], 70.0], 
 [['Apple'], ['Banana'], 41.66666666666667], 
 [['Apple'], ['Mango', 'Banana'], 33.33333333333333], 
 [['Mango', 'Apple'], ['Banana'], 57.14285714285714], 
 [['Apple', 'Banana'], ['Mango'], 80.0], 
 [['Mango', 'Banana'], ['Apple'], 66.66666666666666], 
 [['Mango', 'Apple'], ['Raspberry'], 28.57142857142857
 [['Raspberry', 'Apple'], ['Mango'], 50.0], 
 [['Mango', 'Raspberry'], ['Apple'], 50.0], 
 [['Mango', 'Banana'], ['Raspberry'], 33.33333333333333], 
 [['Mango', 'Raspberry'], ['Banana'], 50.0], 
 [['Raspberry', 'Banana'], ['Mango'], 50.0]
]
The first item-set in arrays shows the products that customers bought and the second shows customers might buy if they bought products in the first item-set. The last element of the array shows the confidence value for that pattern. For example, a customer who buys Mango might buy Apple with %58 confidence. Another example; customer who buys Mango and Banana might buy Apple with 66.6% confidence. If you look carefully at the first two arrays [“Mango”, “Apple” ] and [“Apple”, “Mango”] have different confidence values. Let’s write the support formula for [“Mango” => “Apple”] (confidence of buying Apple after Mango)
( support(["Mango" , "Apple"]) / support(["Mango"]) ) * 100 = 70
and formula of [“Apple” => “Mango”]
( support(["Mango" , "Apple"]) / support(["Apple"]) ) * 100 = 58
Support of [“Mango”, “Apple”] is 7, support of [“Mango”] is 10, and support of [“Apple”] is 12. Although the frequency (support) of [“Apple” “Mango”] is the same in both calculations, they have different confidence values ​​because the frequency of “Apple” and the frequency of “Mango” are different.
You can also convert the result pandas data-frame by using the below codes;

In [ ]:
## Import pandas modules
import pandas as pd

## Create an array with collected baseRddConfidence results
result = baseRddConfidence.collect()

## Create Data Frame
confidenceTable = pd.DataFrame(data = result , columns=["Before", "After" , "Confidence"])

## Show data frame
print(confidenceTable)

### Conclusion
In this article, we learned what is Frequent Pattern Mining and how to apply it over the Apriori algorithm. We took an example case with a data-set and applied the Apriori algorithm on PySpark from scratch. There are also other FPM methods that you can check out such as FPGrowth, Eclat, and so on. After understanding Apriori you can easily understand how other methods work. If you have any questions please feel free to ask.
Hope it was helpful…